In [ ]:
pip install geopandas

In [ ]:
pip install bokeh

In [ ]:
pip install country_converter

In [ ]:
pip install fuzzy_pandas

In [ ]:
pip install plotly

In [ ]:
pip install plotly

In [ ]:
pip install chart_studio

In [ ]:
pip install readme-md

In [13]:
import pandas as pd
import geopandas as gpd
import fiona
import numpy as np
import functools
import country_converter
import pycountry
import json
import chart_studio
import plotly.express as px

In [14]:
data_path = '~/Documents/Uni/qm2g1/data/processed_data/protests_data_filtered.csv'
protest_data =  pd.read_csv(data_path, index_col=1)

#Summarise all protests per country per year

protest_data = protest_data.groupby(['country','year'])['protest'].sum().reset_index(name='protests')

In [15]:
data_path = '~/Documents/Uni/qm2g1/data/maps/country_class.csv'
country_class =  pd.read_csv(data_path, index_col=0)

In [16]:
protest_data = protest_data.merge(country_class, left_on='country', right_on='Country', how='left')

In [17]:
#Download shapefile from Naturalearth
url = "https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_50m_admin_0_countries.geojson"
shapefile_data = gpd.read_file(url)[['sovereignt', 'adm0_a3', 'geometry']]

#to geo jsnon
shapefile_data.to_file("custom_map.json", driver='GeoJSON')

with open('custom_map.json') as f:
  world_map = json.load(f)

In [18]:
# merge shapefile data with protest data
df = protest_data.merge(shapefile_data, left_on = 'country', right_on = 'sovereignt')
df.head(10)
#df.to_excel('control_view.xlsx')

,country,year,protests,Region,sovereignt,adm0_a3,geometry
0,Afghanistan,2000,0,South Asia,Afghanistan,AFG,"POLYGON ((74.89131 37.23164, 74.84023 37.22505..."
1,Afghanistan,2001,0,South Asia,Afghanistan,AFG,"POLYGON ((74.89131 37.23164, 74.84023 37.22505..."
2,Afghanistan,2002,1,South Asia,Afghanistan,AFG,"POLYGON ((74.89131 37.23164, 74.84023 37.22505..."
3,Afghanistan,2003,0,South Asia,Afghanistan,AFG,"POLYGON ((74.89131 37.23164, 74.84023 37.22505..."
4,Afghanistan,2004,0,South Asia,Afghanistan,AFG,"POLYGON ((74.89131 37.23164, 74.84023 37.22505..."
5,Afghanistan,2005,2,South Asia,Afghanistan,AFG,"POLYGON ((74.89131 37.23164, 74.84023 37.22505..."
6,Afghanistan,2006,0,South Asia,Afghanistan,AFG,"POLYGON ((74.89131 37.23164, 74.84023 37.22505..."
7,Afghanistan,2007,0,South Asia,Afghanistan,AFG,"POLYGON ((74.89131 37.23164, 74.84023 37.22505..."
8,Afghanistan,2008,0,South Asia,Afghanistan,AFG,"POLYGON ((74.89131 37.23164, 74.84023 37.22505..."
9,Afghanistan,2009,0,South Asia,Afghanistan,AFG,"POLYGON ((74.89131 37.23164, 74.84023 37.22505..."


In [19]:
#Round off the locations to 2 decimal places (about 1.1 km accuracy)
for i in range(0, len(world_map["features"])):
    for j in range(0,len(world_map["features"][i]['geometry']['coordinates'])):
        try:
            world_map["features"][i]['geometry']['coordinates'][j] = np.round(np.array(world_map["features"][i]['geometry']['coordinates'][j]),2)
        except:
            print(i,j)

7 4
9 1
17 2
106 7
227 2
238 1


In [20]:
df_d = shapefile_data[['sovereignt']]

In [21]:
world_map["features"][0].keys()

dict_keys(['type', 'properties', 'geometry'])

In [22]:
merged = df_d.set_index('sovereignt').join(df.set_index('country'))
merged.reset_index(inplace=True)
merged.head()

,index,year,protests,Region,sovereignt,adm0_a3,geometry
0,Afghanistan,2000.0,0.0,South Asia,Afghanistan,AFG,"POLYGON ((74.89131 37.23164, 74.84023 37.22505..."
1,Afghanistan,2001.0,0.0,South Asia,Afghanistan,AFG,"POLYGON ((74.89131 37.23164, 74.84023 37.22505..."
2,Afghanistan,2002.0,1.0,South Asia,Afghanistan,AFG,"POLYGON ((74.89131 37.23164, 74.84023 37.22505..."
3,Afghanistan,2003.0,0.0,South Asia,Afghanistan,AFG,"POLYGON ((74.89131 37.23164, 74.84023 37.22505..."
4,Afghanistan,2004.0,0.0,South Asia,Afghanistan,AFG,"POLYGON ((74.89131 37.23164, 74.84023 37.22505..."


In [23]:
df = df[df['Region'] == 'Middle East & North Africa']
df.head()

,country,year,protests,Region,sovereignt,adm0_a3,geometry
40,Algeria,2000,0,Middle East & North Africa,Algeria,DZA,"POLYGON ((8.57656 36.93721, 8.59766 36.88389, ..."
41,Algeria,2001,6,Middle East & North Africa,Algeria,DZA,"POLYGON ((8.57656 36.93721, 8.59766 36.88389, ..."
42,Algeria,2002,0,Middle East & North Africa,Algeria,DZA,"POLYGON ((8.57656 36.93721, 8.59766 36.88389, ..."
43,Algeria,2003,0,Middle East & North Africa,Algeria,DZA,"POLYGON ((8.57656 36.93721, 8.59766 36.88389, ..."
44,Algeria,2004,0,Middle East & North Africa,Algeria,DZA,"POLYGON ((8.57656 36.93721, 8.59766 36.88389, ..."


fig = px.choropleth(df, locations="country_code", 
                     color="protests",
                     hover_name="country",
                     hover_data='protests',
                     animation_frame="year",
                     projection="natural earth",
                     color_continuous_scale = px.colors.diverging.Temps
                     )

In [ ]:
#mapbox map

fig = px.choropleth_mapbox(
    df,
    geojson=world_map,
    locations='adm0_a3',
    featureidkey="properties.adm0_a3",
    color=df['protests'],
    color_continuous_scale = px.colors.diverging.Temps,
    hover_name='country',
    animation_frame="year",
 #   hover_data={'country': False, 'year': False, 'protests': True, 'Region' : False, 'sovereignt':False, 'adm0_a3': False, 'geometry':False},
    mapbox_style='open-street-map',
    zoom=4,
    center={'lat': 29, 'lon': 22},
    opacity=0.6
)


In [ ]:
# Set Layout

fig.update_layout(
    title_text="Number of Protests",
    margin={'r':0,'t':0,'l':0,'b':0},
    coloraxis_colorbar={
        'title':'Protests'      
    }
)


#fig.update_geos(fitbounds='locations')

TypeError: Object of type Polygon is not JSON serializable

TypeError: Object of type Polygon is not JSON serializable

# Upload Plot to Plotly

In [ ]:
username = 'maierhoj' # your username 
api_key = 'PfelhyGBHvZTo4S265iM' # your api key - go to profile > settings > regenerate  key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

import chart_studio.plotly as py
py.plot(fig, filename = 'protests_map', auto_open=True)

In [ ]:
import country_converter as coco